In [ ]:
import requests
from bs4 import BeautifulSoup

from datetime import date, timedelta

URL = 'https://www.tribuneindia.com/'

def all_sundays(year):
    # January 1st of the given year
    dt = date(year, 1, 1)
    # First Sunday of the given year
    dt += timedelta(days=6 - dt.weekday())
    while dt.year == year:
        yield dt
        dt += timedelta(days=7)

def ads_from_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    ad_1 = soup.find_all('table')
    brides_wanted = []
    grooms_wanted = []

    for ad in ad_1:
        if "Brides Wanted" in str(ad):
            ad_list = ad.find_all('p')
            for i in range(len(ad_list)):
                brides_wanted.append(' '.join(ad_list[i].get_text().replace('\r', ' ').replace('\n', ' ').rstrip().split()))
                #' '.join(ad_2[i].get_text().replace('\r', ' ').replace('\n', ' ').rstrip().split())
        if "Grooms Wanted" in str(ad):
            ad_list = ad.find_all('p')
            for i in range(len(ad_list)):
                grooms_wanted.append(' '.join(ad_list[i].get_text().replace('\r', ' ').replace('\n', ' ').rstrip().split()))

    f1.write("b'grooms wanted'\n")
    for groom in brides_wanted:
        groom = groom.encode('ascii', 'ignore')
        f1.write(str(groom)+"\n")

    f1.write("b'brides wanted'\n")
    for bride in grooms_wanted:
        f1.write(str(bride.encode('ascii', 'ignore'))+"\n")

if __name__ == '__main__':
   
    for year in range(1998, 2000):
        f1 = open("E:/Graduate stuff/Machine Learning and Deep Learning/matad/Advertisments by years/advertisments"+str(year)+".txt", "w")

        for sunday in all_sundays(year):
            d = sunday.strftime('%y%b%d')
            if year == 2018 and d == '18dec16':
                break
            url = URL+str(year)+'/'+str(d.lower())+'/class.htm'
            print(url)
            ads_from_page(url)


In [ ]:
def grooms_brides(filename):
    grooms = []
    brides = []
    groom = 0
    with open(filename,"r") as f:
        for line in f:
            if "brides" in line.lower():
                groom = 0
                continue
            elif "grooms" in line.lower():
                groom = 1
                continue
            elif groom == 0:
                grooms.append(line[2:-2])
            else:
                brides.append(line[2:-2])

    return grooms, brides

brides = []
grooms = []

if __name__ == '__main__':

    for year in range(1998, 2000):
        groomsy, bridesy = grooms_brides("E:/Graduate stuff/Machine Learning and Deep Learning/matad/Advertisments by years/advertisments"+str(year)+".txt")
        brides.append(bridesy)
        grooms.append(groomsy)



In [ ]:
ad2013_brides = "E:/Graduate stuff/Machine Learning and Deep Learning/matad/Corpus/Brides (Grooms wanted)/Brides_tribuneindia_2013.txt"
brides2013 = []

f = open(ad2013_brides, encoding = "utf8")
line = "\n"

while line:
    line = f.readline()
    if str(line)!= "b'\\n'":
        brides2013.append(line)   
f.close()

brides.append(brides2013)


In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer("english")
stopwords = nltk.corpus.stopwords.words('english')
context = ["suitab","match","prefer","girl","boy","man","from","suitabl","suitable","preferr","preferred","gmail"]
context1 = ['born', 'contact', 'email', 'famili', 'girl', 'gmail.com', 'match', 'prefer', 'profession', 'profession qualifi', 'qualifi', 'settl','suitabl', 'suitabl match', 'work', 'yahoo.com']
stopwords.extend(context)
stopwords.extend(context1)

In [ ]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for year in range(1998,2001):
    for bride in brides[year-1998]:
        allwords_stemmed = tokenize_and_stem(bride) #for each item in 'synopses', tokenize/stem
        totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
        allwords_tokenized = tokenize_only(bride)
        totalvocab_tokenized.extend(allwords_tokenized)
    if year == 2000:
        break
    for groom in grooms[year-1998]:
        allwords_stemmed = tokenize_and_stem(groom) #for each item in 'synopses', tokenize/stem
        totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
        allwords_tokenized = tokenize_only(groom)
        totalvocab_tokenized.extend(allwords_tokenized)
        

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print("total vocab tokenized: ")
print(len(totalvocab_tokenized))
print("total vocab stemmed: ")
print(len(totalvocab_stemmed))
print("total vocab: ")
#print(vocab_frame)
#print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

In [ ]:
print(vocab_frame.head())

###### This is taking into account the English stopwords..... That will cause another issue

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.93, max_features=200000,
                                 min_df=0.08, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix_brides = tfidf_vectorizer.fit_transform(brides[0]) #fit the vectorizer to brides
#%time tfidf_matrix_grooms = tfidf_vectorizer.fit_transform(grooms[0]) #fit the vectorizer to grooms
%time tfidf_matrix_brides2013 = tfidf_vectorizer.fit_transform(brides[2]) #fit the vectorizer to brides 2013


In [ ]:
terms = tfidf_vectorizer.get_feature_names()
print(terms)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist_brides = 1 - cosine_similarity(tfidf_matrix_brides2013)
dist_brides1999 = 1 - cosine_similarity(tfidf_matrix_brides)

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix_brides2013)

clusters = km.labels_.tolist()

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km1 = KMeans(n_clusters=num_clusters)

%time km1.fit(tfidf_matrix_brides)

clusters1 = km1.labels_.tolist()

In [ ]:
films = {'ads':brides[2], 'cluster':clusters}
frame = pd.DataFrame(films, index = [clusters])
frame['cluster'].value_counts()

In [ ]:
films1 = {'ads':brides[0], 'cluster':clusters1}
frame = pd.DataFrame(films1, index = [clusters1])
frame['cluster'].value_counts()

In [ ]:

from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 6 with n words per cluster
        #print(terms[ind])
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
"""
    print("Cluster %d titles:" % i, end='')
    for title in frame.iloc[i]['ad'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
"""    
print()
print()